#LSTM Char Level

In [0]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

file = open('gdrive/My Drive/Redes Neurais/lfv_texts_plain.txt').read()
file_len = len(file)
print('File_len = ', file_len)

all_characters = ''.join(set(file))
n_characters = len(all_characters)

print(all_characters, n_characters)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
File_len =  1298370
stìM(UÕvz5ü)u”çî‘
1XafÉèq 4gQ3-…d~m;̧Áõ"̂BF&h/l7x–wP.ôEeã’̈õV*?C9—ÇÍyn60IR L8âjú̀2ó$―rbiÓ!́éYTÃ“SGDN,°Oáà@kc'`êÀW%íA:KZpHJ 123


In [2]:
chunk_len = 200

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

NameError: ignored

In [0]:
# Converte string para uma lista de inteiros

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
      try:
        tensor[c] = all_characters.index(string[c])
      except:
        print(c)
        raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([ 20,  87, 108,  99,  55,  43])


In [0]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return (Variable(torch.zeros(self.n_layers, 1, self.hidden_size)), 
                Variable(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
      
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [0]:
def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
n_epochs = 10000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(), example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[Epoch: %d \t Progress: %d%% \t Loss: %.4f \t Time: %s]' % (epoch, epoch / n_epochs * 100, loss, time_since(start)))
        print(evaluate('Os', 100), '\n')


[Epoch: 100 	 Progress: 1% 	 Loss: 2.5073 	 Time: 0m 43s]
Os áu sonatara dne orCa cTomó do cô erie quses cándes dacaM om sane nos nua des. fola aas danai. ga 

[Epoch: 200 	 Progress: 2% 	 Loss: 2.4730 	 Time: 1m 27s]
Os comem o se coscia sálbas uhaca sa os monheno le Mado e pao porá. tadeo rasos, e coneitia do a elo  

[Epoch: 300 	 Progress: 3% 	 Loss: 2.2402 	 Time: 2m 11s]
Osit,
Tãeçs, eu mainte de fom ntom. mago, pravalava cobinte o mé forto, não mão tentas ale erão  

[Epoch: 400 	 Progress: 4% 	 Loss: 2.0386 	 Time: 2m 55s]
Osse seminada com vo-tes tazeria ão Pesberendo, veme vigarantanco, dintadabado tor a) não comas nara 

[Epoch: 500 	 Progress: 5% 	 Loss: 1.9627 	 Time: 3m 39s]
Os? Mas hame gais o sous da mul, oucite e neferrem não comego ente que espoi a ques de que porpreu en 

[Epoch: 600 	 Progress: 6% 	 Loss: 2.5347 	 Time: 4m 23s]
Os de istalar a suncado.
― ― de Haté como, Brista gura no de Tamartos da as ina entre porrama cungobr 

[Epoch: 700 	 Progress

In [0]:
print(evaluate('O psicanalista de', 5500), '\n')

O psicanalista de terceiros estarem para virar assimentos pensados do chegonados poltores para carne, e uma lintar assim. Pode não resuldade. "Desculpado
que eu tendo se pedem nos generado que o que você sia da frença.
"Ai se forma a litíngua desligada para outra casa. E eu dentre. Muito dissera ou simpnes. No adtes mais é a mão de pessoa que está sofado de cibelando, pudessem doram descontrovro parcimento, desenvolve os divórcios em deve cumbina a minha Cordão sombinho de preciso peritor, duas nossa encondiçada pelo Tacgurair, apareceram do seu solentado. E quando eu calmentando controlents e está de peus gerrados, o Márcio é que assustários existem para ter lá e respir tudo a distâner. Ainda ainda seria nestímina ou tempo sem jogador para conventara precisa vendência, como entrou a sua, como seu lado. Não saber com cruzar, ou de falar, Maral domento, a Vaneral. Não moria não me convencido? Apenas contar um perto do de-olonizada por um desencontro, numa razão de 

# LSTM Word Level


In [12]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
symbols = set([",", ".", "!", "?", "*", ")", "(", '"', "\\", "/", ":", "...", "”", "“"])

from google.colab import drive
drive.mount('/content/gdrive')

file = open('gdrive/My Drive/Redes Neurais/lfv_texts_plain.txt', encoding='utf-8').read().lower().replace('\n', ' \n ')
for symbol in symbols:
    file = file.replace(symbol, " " + symbol + " ")

file = [w for w in file.split(' ') if w.strip() != '' or w == '\n']
file_len = len(file)
print('file_len =', file_len)

all_words = list(set(file))
n_words = len(all_words)

print(all_words, n_words)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
file_len = 277409
['atiravam-se', 'interessado', 'pré-brasileiro', 'nomes', 'intimidades', 'traços', 'curiosidade', 'instigado', 'espanhóis', 'matá-lo', 'xeque', 'lãs', 'proteínas', 'primitivo', 'desnecessário', 'limo', 'impacientou', 'recomendações', 'suíços', 'vislumbravam', 'desconfiado', 'digitais', 'turística', 'calculando', 'rei', 'incluído', 'clint', 'alerta', 'dispensei', 'mágicas', 'dedica-se', 'entusiasmara', 'macho', 'choque', 'celinha'

In [13]:
# Converte lista de string para uma lista de inteiros

def string_tensor(strings):
    tensor = torch.zeros(len(strings)).long()
    for i in range(len(strings)):
      try:
        tensor[i] = all_words.index(strings[i])
      except:
        print(i)
        raise
    return Variable(tensor)

print(string_tensor(["de", "bagé", "razão"]))

tensor([ 7485, 15889,  7476])


In [0]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = string_tensor(chunk[:-1])
    target = string_tensor(chunk[1:])
    return inp, target

In [0]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str=['a'], predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    prime_input = string_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_string = all_words[top_i]
        predicted.append(predicted_string)
        inp = string_tensor([predicted_string]).cuda()

    return ' '.join(predicted)

In [14]:
n_epochs = 10000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_words, hidden_size, n_words, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate(['os'], 100), '\n')

 59s (100 1%) 6.8565]
os cantada quem dona . , . , que afinal : com igual , 
 e de , de , que ― a , proa morte revista grande ! gritaria arte que , mais portugal . . , , , eu que a a de , viagem . entre a 
 alta ― da mulheres de que . fazem de os para . , as . grupo a de que o sempre a que garrafa um escondia que , como do brasileiro pura de " , ! únicos no e , , atrás , , de da no que por e 

 59s (200 2%) 6.9703]
os empenhado derrubando . . com num , por escravagismo de família . , . : e pedidos o aparece . 
 a dela 
 , no corrupto . que quis era . 
 e era menos um nunca e o volto estadão o pensei chegaram , , , e reforçar mas , , na apresentados a cocoricó em insegurança a quem , . ― . 
 . ― . ? é uma recheando , peito . 
 
 , vez borbulhante mundo , mesmo da se desocupara , chegar cá " na lhe talvez " papal ― o caçula quando 

 59s (300 3%) 6.4121]
os mais estancava . ― não o o 18 . punho novo em carregamos , que em casamento . ele ou primeiro se destruída para congelado

In [15]:
# Com a rede treinada, gerar um texto longo

print(evaluate(['o', 'psicanalista', 'de'], 5500), '\n')

o psicanalista de que o brasil não podia haver mais de dez ou de alguém ou sem canivete . o rosto , o mantém . o dr . desgraçadamente : 
 ― mudanças ? 
 ― não . . . 
 ― o quê ? 
 ― se eu gosto que me dei . 
 ― então é você , é uma merda ? 
 ― eu também pode deixar , saddam , não tenho . 
 ― não . 
 ― eu sei que sabe o seu filho ! 
 ― jura , dr . 
 ― você não lembra ? 
 ― a mulher não usavam infeliz , anastácia beija 
 terminar . 
 ― assim ? 
 ― é . na verdade é que nossa solidão . 
 uma ano descobriram que o primeiro era para a gente me vai me dar uma boa mão . ― nós só sem uma identidade . 
 acupuntura . o que é que é isso ? ! . . . 
 mas quem não teria mais nada ? um homem não queria fazer em mulher , em algum lugar . se não sei que não era o livro . 
 ― desde o que você fez ? 
 ― não . quem é ? 
 ― você vai acaba . 
 ― eu . . . 
 ― eu sei de que você tem muito mais pronto que não é formidável , entende ? 
 ― você não sabe e eu . 
 ― luciana , e

# **Considerações:**



*   Foi verificado uma diferença pequena entre LSTM e GRU. Caso formos ver no treinamento desses dois modelos, a GRU terminava uma época mais rápido do que a LSTM,  pórem as duas conseguiram gerar resultados interessantes e legíveis.
*   Para realizar o treinamento em nível de palavra, foi necessário fazer algumas adaptações, como criar uma lista com todas as palavras existentes no texto e converter cada palavra em um tensor diferente, para poder colocar como entrada na rede.
* Em nível de palavra, é possível notar que teve uma certa coerência, mesmo que ainda não tenha um total sentido, diferente da LSTM treinada a nível de caractere, onde palavras foram simplesmente jogadas no texto gerado.

